In [4]:
import numpy as np
import anndata as ad
import scipy.io as scIO
import scipy.sparse as scs
import pandas as pd
from time import time

In [16]:
np.__version__

'1.21.5'

# Import and Explore

In [17]:
idir = '/Users/adityagorla/Desktop/STATS254/opMultiome10x'

In [18]:
adata_gex = ad.read_h5ad(f"{idir}/multiome_gex_processed_training.h5ad")
adata_atac = ad.read_h5ad(f"{idir}/multiome_atac_processed_training.h5ad")

In [19]:
print(f"The GEX data has {adata_gex.n_obs} observations and {adata_gex.n_vars} features.")
print(f"The ATAC data has {adata_atac.n_obs} observations and {adata_atac.n_vars} features.")

The GEX data has 42492 observations and 13431 features.
The ATAC data has 42492 observations and 116490 features.


In [6]:
adata_gex

AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'cell_type', 'pseudotime_order_GEX', 'batch', 'pseudotime_order_ATAC', 'is_train'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'dataset_id', 'organism'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [7]:
adata_atac

AnnData object with n_obs × n_vars = 42492 × 116490
    obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'cell_type', 'pseudotime_order_ATAC', 'batch', 'pseudotime_order_GEX', 'is_train'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism', 'sample_pm_varnames'
    obsm: 'gene_activity', 'lsi_full', 'lsi_red', 'umap'
    layers: 'counts'

In [13]:
len(set(adata_atac.obs["cell_type"]))

22

In [14]:
len(set(adata_gex.obs["cell_type"]))

22

In [12]:
set(adata_atac.obs["cell_type"]).intersection(set(adata_gex.obs["cell_type"]))

{'B1 B',
 'CD14+ Mono',
 'CD16+ Mono',
 'CD4+ T activated',
 'CD4+ T naive',
 'CD8+ T',
 'CD8+ T naive',
 'Erythroblast',
 'G/M prog',
 'HSC',
 'ID2-hi myeloid prog',
 'ILC',
 'Lymph prog',
 'MK/E prog',
 'NK',
 'Naive CD20+ B',
 'Normoblast',
 'Plasma cell',
 'Proerythroblast',
 'Transitional B',
 'cDC2',
 'pDC'}

In [27]:
adata_gex.X

<42492x13431 sparse matrix of type '<class 'numpy.float64'>'
	with 55690186 stored elements in Compressed Sparse Column format>

In [36]:
adata_atac.var.index

Index(['chr1-9776-10668', 'chr1-180726-181005', 'chr1-181117-181803',
       'chr1-191133-192055', 'chr1-267562-268456', 'chr1-629497-630394',
       'chr1-633515-634474', 'chr1-778276-779191', 'chr1-816868-817761',
       'chr1-822804-823597',
       ...
       'GL000195.1-137376-138301', 'GL000219.1-39933-40839',
       'GL000219.1-42172-43054', 'GL000219.1-44703-45584',
       'GL000219.1-45726-46450', 'GL000219.1-90062-90937',
       'GL000219.1-99257-100160', 'KI270726.1-27152-28034',
       'KI270713.1-21434-22336', 'KI270713.1-29629-30491'],
      dtype='object', length=116490)

In [80]:
adata_gex.var

,gene_ids,feature_types,genome
AL627309.5,ENSG00000241860,GEX,GRCh38
LINC01409,ENSG00000237491,GEX,GRCh38
LINC01128,ENSG00000228794,GEX,GRCh38
NOC2L,ENSG00000188976,GEX,GRCh38
KLHL17,ENSG00000187961,GEX,GRCh38
...,...,...,...
MT-ND5,ENSG00000198786,GEX,GRCh38
MT-ND6,ENSG00000198695,GEX,GRCh38
MT-CYB,ENSG00000198727,GEX,GRCh38
AL592183.1,ENSG00000273748,GEX,GRCh38


In [33]:
sum(adata_atac.obs['batch'] != adata_gex.obs['batch'])

0

In [72]:
adata_atac.X

<42492x116490 sparse matrix of type '<class 'numpy.float64'>'
	with 143812825 stored elements in Compressed Sparse Column format>

In [47]:
adata_gex.obs

,pct_counts_mt,n_counts,n_genes,size_factors,phase,cell_type,pseudotime_order_GEX,batch,pseudotime_order_ATAC,is_train
TAGTTGTCACCCTCAC-1-s1d1,1.061008,1508.0,1022.0,0.453484,S,Naive CD20+ B,NaN,s1d1,NaN,True
CTATGGCCATAACGGG-1-s1d1,0.604230,1655.0,1081.0,0.455631,G2M,CD14+ Mono,NaN,s1d1,NaN,True
CCGCACACAGGTTAAA-1-s1d1,0.650069,7230.0,3304.0,2.435348,G2M,CD8+ T,NaN,s1d1,NaN,True
TCATTTGGTAATGGAA-1-s1d1,0.812274,1108.0,793.0,0.347226,G2M,CD8+ T,NaN,s1d1,NaN,True
ACCACATAGGTGTCCA-1-s1d1,1.674770,1851.0,1219.0,0.534205,G2M,CD16+ Mono,NaN,s1d1,NaN,True
...,...,...,...,...,...,...,...,...,...,...
ATTCACTTCCTGCGAA-14-s3d7,0.000000,1475.0,967.0,0.636243,G2M,CD8+ T,NaN,s3d7,NaN,True
GCTCTGTTCTGCAAGT-14-s3d7,0.014362,6963.0,3237.0,3.019859,S,G/M prog,NaN,s3d7,NaN,True
GCTGAGGAGTGAGCGG-14-s3d7,0.068399,1462.0,876.0,0.495807,G2M,Erythroblast,0.791014,s3d7,0.828536,True
TACTGAGGTTCGCTCA-14-s3d7,0.000000,2808.0,1584.0,1.125766,G2M,CD14+ Mono,NaN,s3d7,NaN,True


In [46]:
sum(adata_atac.obs['cell_type'] != adata_gex.obs['cell_type'])

0

In [48]:
adata_atac.obs

,nCount_peaks,atac_fragments,reads_in_peaks_frac,blacklist_fraction,nucleosome_signal,cell_type,pseudotime_order_ATAC,batch,pseudotime_order_GEX,is_train
TAGTTGTCACCCTCAC-1-s1d1,4031.0,5400,0.746481,0.003473,0.642468,Naive CD20+ B,NaN,s1d1,NaN,True
CTATGGCCATAACGGG-1-s1d1,8636.0,19266,0.448251,0.003126,1.220679,CD14+ Mono,NaN,s1d1,NaN,True
CCGCACACAGGTTAAA-1-s1d1,4674.0,6177,0.756678,0.001284,0.692573,CD8+ T,NaN,s1d1,NaN,True
TCATTTGGTAATGGAA-1-s1d1,2803.0,4019,0.697437,0.000714,0.633838,CD8+ T,NaN,s1d1,NaN,True
ACCACATAGGTGTCCA-1-s1d1,1790.0,2568,0.697040,0.003352,0.727660,CD16+ Mono,NaN,s1d1,NaN,True
...,...,...,...,...,...,...,...,...,...,...
ATTCACTTCCTGCGAA-14-s3d7,1349.0,3065,0.440131,0.000000,0.963134,CD8+ T,NaN,s3d7,NaN,True
GCTCTGTTCTGCAAGT-14-s3d7,8312.0,20679,0.401954,0.000241,0.756219,G/M prog,NaN,s3d7,NaN,True
GCTGAGGAGTGAGCGG-14-s3d7,2674.0,6903,0.387368,0.000000,0.941300,Erythroblast,0.828536,s3d7,0.791014,True
TACTGAGGTTCGCTCA-14-s3d7,3920.0,8229,0.476364,0.000510,0.788606,CD14+ Mono,NaN,s3d7,NaN,True


In [51]:
for elm in set(adata_atac.obs['batch']):
    print(f"{elm} :{list(adata_atac.obs['batch']).count(elm)}")
# Use s2 as train


s1d1 :5616
s3d7 :6094
s3d10 :3909
s2d5 :4395
s1d3 :3875
s1d2 :6069
s3d3 :1496
s2d4 :5456
s2d1 :3811
s3d6 :1771


In [52]:
type(adata_gex.X)

scipy.sparse.csc.csc_matrix

(array([    0,     0,     0, ..., 42491, 42491, 42491], dtype=int32),
 array([   45,    68,    70, ..., 13328, 13356, 13391], dtype=int32))

In [59]:
combArray = scs.hstack([adata_gex.X, adata_atac.X])

In [60]:
combArray

<42492x129921 sparse matrix of type '<class 'numpy.float64'>'
	with 199503011 stored elements in Compressed Sparse Column format>

In [61]:
del(combArray)

In [74]:
adata_gex.var.gene_ids

AL627309.5    ENSG00000241860
LINC01409     ENSG00000237491
LINC01128     ENSG00000228794
NOC2L         ENSG00000188976
KLHL17        ENSG00000187961
                   ...       
MT-ND5        ENSG00000198786
MT-ND6        ENSG00000198695
MT-CYB        ENSG00000198727
AL592183.1    ENSG00000273748
AC240274.1    ENSG00000271254
Name: gene_ids, Length: 13431, dtype: category
Categories (13431, object): ['ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938', ..., 'ENSG00000288093', 'ENSG00000288107', 'ENSG00000288156', 'ENSG00000288380']

In [97]:
adata_atac.uns['sample_pm_varnames']

10000

In [102]:
adata_atac.obsm['gene_activity']

<42492x19039 sparse matrix of type '<class 'numpy.float64'>'
	with 238859138 stored elements in Compressed Sparse Column format>

# Build full MTX w/ annots

### Full X data

In [86]:
fullANNOT = pd.DataFrame(data = {'cellID' : list(adata_gex.obs.index),
                                 'cellLabel' : list(adata_gex.obs['cell_type'])})
fullANNOT.head(10)

,cellID,cellLabel
0,TAGTTGTCACCCTCAC-1-s1d1,Naive CD20+ B
1,CTATGGCCATAACGGG-1-s1d1,CD14+ Mono
2,CCGCACACAGGTTAAA-1-s1d1,CD8+ T
3,TCATTTGGTAATGGAA-1-s1d1,CD8+ T
4,ACCACATAGGTGTCCA-1-s1d1,CD16+ Mono
5,TGGATTGGTTTGCGAA-1-s1d1,B1 B
6,GTGAGCGAGTAAAGGT-1-s1d1,Lymph prog
7,GACTTAGGTTGCGCGA-1-s1d1,CD4+ T naive
8,GCCTTACTCGTTACAA-1-s1d1,CD4+ T activated
9,GTAAGGTCAATAACCT-1-s1d1,B1 B


In [88]:
fullANNOT['batch'] = list(adata_gex.obs.batch)
fullANNOT['phase'] = list(adata_gex.obs.phase)
fullANNOT.head(10)

,cellID,cellLabel,batch,phase
0,TAGTTGTCACCCTCAC-1-s1d1,Naive CD20+ B,s1d1,S
1,CTATGGCCATAACGGG-1-s1d1,CD14+ Mono,s1d1,G2M
2,CCGCACACAGGTTAAA-1-s1d1,CD8+ T,s1d1,G2M
3,TCATTTGGTAATGGAA-1-s1d1,CD8+ T,s1d1,G2M
4,ACCACATAGGTGTCCA-1-s1d1,CD16+ Mono,s1d1,G2M
5,TGGATTGGTTTGCGAA-1-s1d1,B1 B,s1d1,G2M
6,GTGAGCGAGTAAAGGT-1-s1d1,Lymph prog,s1d1,S
7,GACTTAGGTTGCGCGA-1-s1d1,CD4+ T naive,s1d1,G2M
8,GCCTTACTCGTTACAA-1-s1d1,CD4+ T activated,s1d1,S
9,GTAAGGTCAATAACCT-1-s1d1,B1 B,s1d1,G2M


In [89]:
fullANNOT.shape

(42492, 4)

In [83]:
fullFEATS = pd.DataFrame(data = { 'featID' : list(adata_gex.var.index)+list(adata_atac.var.index),
                                 'featType': list(adata_gex.var.feature_types)+list(adata_atac.var.feature_types)})
fullFEATS.tail(10)

,featID,featType
129911,GL000195.1-137376-138301,ATAC
129912,GL000219.1-39933-40839,ATAC
129913,GL000219.1-42172-43054,ATAC
129914,GL000219.1-44703-45584,ATAC
129915,GL000219.1-45726-46450,ATAC
129916,GL000219.1-90062-90937,ATAC
129917,GL000219.1-99257-100160,ATAC
129918,KI270726.1-27152-28034,ATAC
129919,KI270713.1-21434-22336,ATAC
129920,KI270713.1-29629-30491,ATAC


In [82]:
fullFEATS.shape

(129921, 2)

In [90]:
combArray = scs.hstack([adata_gex.X, adata_atac.X])
combArray.shape

(42492, 129921)

In [91]:
scIO.mmwrite('opmultiome_GEXATAC.quantitative.mtx', combArray)
del(combArray)

In [92]:
fullANNOT.to_csv('opmultiome_GEXATAC.quantitative.annot.tsv', sep="\t", index=False)

In [94]:
fullFEATS.to_csv('opmultiome_GEXATAC.quantitative.features.tsv', sep="\t", index=False)

### GEX and AS data

In [99]:
gexasFEATS = pd.DataFrame(data = { 'featID' : list(adata_gex.var.index)+list(adata_atac.uns['gene_activity_var_names']),
                                 'featType': list(adata_gex.var.feature_types)+list(['ATACas']*len(adata_atac.uns['gene_activity_var_names']))})
gexasFEATS.tail(10)

,featID,featType
32460,BPY2,ATACas
32461,DAZ1,ATACas
32462,DAZ2,ATACas
32463,PRYP3,ATACas
32464,CDY1B,ATACas
32465,BPY2B,ATACas
32466,DAZ3,ATACas
32467,DAZ4,ATACas
32468,BPY2C,ATACas
32469,CDY1,ATACas


In [100]:
gexasFEATS.shape

(32470, 2)

In [103]:
combArray = scs.hstack([adata_gex.X, adata_atac.obsm['gene_activity']])
combArray.shape

(42492, 32470)

In [104]:
scIO.mmwrite('opmultiome_GEXATACas.quantitative.mtx', combArray)
del(combArray)

In [105]:
fullANNOT.to_csv('opmultiome_GEXATACas.quantitative.annot.tsv', sep="\t", index=False)

In [106]:
gexasFEATS.to_csv('opmultiome_GEXATACas.quantitative.features.tsv', sep="\t", index=False)

### Site 2 only

In [5]:
s2_cells = adata_gex.obs_names[(adata_gex.obs["batch"] =="s2d4") | (adata_gex.obs["batch"] == "s2d5") | (adata_gex.obs["batch"] == "s2d1")]

In [6]:
s2_gex = adata_gex[s2_cells]
s2_atac = adata_atac[s2_cells]

In [7]:
s2ANNOT = pd.DataFrame(data = {'cellID' : list(s2_gex.obs.index),
                                 'cellLabel' : list(s2_gex.obs['cell_type'])})
s2ANNOT['batch'] = list(s2_gex.obs.batch)
s2ANNOT['phase'] = list(s2_gex.obs.phase)
s2ANNOT.head(10)

,cellID,cellLabel,batch,phase
0,ACGTTACAGGCATTAC-4-s2d1,CD16+ Mono,s2d1,S
1,GGTGATTTCGCTAGAT-4-s2d1,Erythroblast,s2d1,S
2,ACAGGATCACTAAGAA-4-s2d1,cDC2,s2d1,G2M
3,CGCTACTTCATCCACC-4-s2d1,CD14+ Mono,s2d1,S
4,ATACCGGTCATGTGGT-4-s2d1,NK,s2d1,G2M
5,GGTAGGAGTACAAAGA-4-s2d1,Erythroblast,s2d1,G2M
6,GGAATCTTCCTCAGTC-4-s2d1,CD14+ Mono,s2d1,G2M
7,ATTCAACCACCACAAC-4-s2d1,Erythroblast,s2d1,G2M
8,CCCTGGACAGCACGAA-4-s2d1,Erythroblast,s2d1,G2M
9,CACTTAAAGGATCACT-4-s2d1,MK/E prog,s2d1,G2M


In [8]:
s2ANNOT.shape

(13662, 4)

In [9]:
s2FEATS = pd.DataFrame(data = { 'featID' : list(s2_gex.var.index)+list(s2_atac.var.index),
                                 'featType': list(s2_gex.var.feature_types)+list(s2_atac.var.feature_types)})
s2FEATS.tail(10)

,featID,featType
129911,GL000195.1-137376-138301,ATAC
129912,GL000219.1-39933-40839,ATAC
129913,GL000219.1-42172-43054,ATAC
129914,GL000219.1-44703-45584,ATAC
129915,GL000219.1-45726-46450,ATAC
129916,GL000219.1-90062-90937,ATAC
129917,GL000219.1-99257-100160,ATAC
129918,KI270726.1-27152-28034,ATAC
129919,KI270713.1-21434-22336,ATAC
129920,KI270713.1-29629-30491,ATAC


In [10]:
s2FEATS.shape

(129921, 2)

In [118]:
combArray = scs.hstack([s2_gex.X, s2_atac.X])
combArray.shape

(13662, 129921)

In [119]:
scIO.mmwrite('opmultiome_GEXATAC_s2.quantitative.mtx', combArray)
del(combArray)

In [11]:
s2ANNOT.to_csv('opmultiome_GEXATAC_s2.quantitative.annot.tsv', sep="\t", index=False)

In [12]:
s2FEATS.to_csv('opmultiome_GEXATAC_s2.quantitative.features.tsv', sep="\t", index=False)

In [20]:
adata_gex.var.index

Index(['AL627309.5', 'LINC01409', 'LINC01128', 'NOC2L', 'KLHL17', 'ISG15',
       'C1orf159', 'SDF4', 'B3GALT6', 'UBE2J2',
       ...
       'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5', 'MT-ND6',
       'MT-CYB', 'AL592183.1', 'AC240274.1'],
      dtype='object', length=13431)

In [8]:
bh = ['SDF4', 'AL627309.5', 'AC240274.1', 'MT-CO3', 'MT-ND5', 'UBE2J2']

In [16]:
gex_subset = np.zeros((1983, 11270))
rnd_mat = np.random.rand(1983, 11270)

In [20]:
st = time()
for i in range(1983):
    gn = i%6
    for j in range(11270):
        cl = j%6
        gex_subset[i,j] = rnd_mat[gn, cl]
print(time()- st)

4.142457008361816
